In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
def to_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d+', cell)
    if not _:
        return cell
    return _[0]

In [4]:
def to_date(cell):
    if not isinstance(cell, str):
        return cell
    return '-'.join(re.findall(r'\d+', cell))

In [5]:
df = pd.read_excel('赛题1数据材料/赛题1数据集/工商基本信息表.xlsx', index_col='企业编号')
df['经营状态'][df['经营状态'] != '迁出'] = '存续（在营、开业、在册）'
df['行业小类（代码）'] = df['行业小类（代码）'].map(to_number)
df['城市代码'] = df['城市代码'].map(lambda x: x % 100)
df['经营期限自'] = df['经营期限自'].map(to_date)
df['发照日期'] = df['发照日期'].map(to_date)
df

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,注册资本（万元）,注册资本币种(正则),成立日期,经营状态,行业大类（代码）,行业小类（代码）,类型,省份代码,城市代码,地区代码,是否上市,经营期限自,经营期限至,登记机关区域代码,发照日期,员工人数,注销原因,注销时间
企业编号,,,,,,,,,,,,,,,,,,
1001,746.138857,人民币,1992-10-19,存续（在营、开业、在册）,J,66,民营,31,0,310000,Y,1992-10-19,NaN,310100.0,1992-10-19,19.0,NaN,NaN
1002,82.772800,人民币,2000-09-19,存续（在营、开业、在册）,O,80,民营,44,0,440000,Y,2000-09-19,NaN,440000.0,2017-12-21,12.0,NaN,NaN
1003,80.000000,人民币,1999-07-21,存续（在营、开业、在册）,C,36,民营,42,0,420000,Y,1999-07-21,NaN,420000.0,2018-10-22,11.0,NaN,NaN
1004,40.291301,人民币,1997-10-15,存续（在营、开业、在册）,F,52,民营,11,1,110105,Y,1997-10-15,NaN,100000.0,2019-02-19,15.0,NaN,NaN
1005,227.417928,人民币,1999-08-31,存续（在营、开业、在册）,L,72,民营,11,1,110102,Y,1999-08-31,NaN,110100.0,2019-01-31,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,61.913480,人民币,1997-12-09,存续（在营、开业、在册）,C,38,民营,44,3,440300,Y,1997-12-09,NaN,440306.0,2018-11-27,9.0,NaN,NaN
3997,30.621000,-,1998-03-10,存续（在营、开业、在册）,L,72,民营,37,2,370200,Y,1998-03-10,NaN,370200.0,2019-02-02,14.0,NaN,NaN
3998,11.626270,人民币,2000-05-25,存续（在营、开业、在册）,I,65,民营,32,0,320000,Y,2000-05-25,NaN,320000.0,2018-07-11,10.0,NaN,NaN


## 筛选数据

In [44]:
registered_capital_ten_thousand_yuan = df[['注册资本（万元）']].groupby('企业编号').mean().T
registered_capital_ten_thousand_yuan = registered_capital_ten_thousand_yuan.reindex(columns=list(range(1001, 4001)))
registered_capital_ten_thousand_yuan

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
注册资本（万元）,746.138857,82.7728,80.0,40.291301,227.417928,77.078338,1823.401306,608.015338,66.3444,512.907466,...,21.98016,18.100696,19.068202,17.20224,23.4,61.91348,30.621,11.62627,19.368798,12.107039


In [6]:
registered_capital_currency_regular = pd.get_dummies(df['注册资本币种(正则)'].replace('-', None), prefix='注册资本币种(正则)').groupby('企业编号').mean().T
registered_capital_currency_regular = registered_capital_currency_regular.reindex(columns=list(range(1001, 4001))).fillna(0)
registered_capital_currency_regular

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
注册资本币种(正则)_人民币,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
注册资本币种(正则)_美元,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [8]:
date_of_establishment = df[['成立日期']].applymap(to_day).groupby('企业编号').mean().T
date_of_establishment = date_of_establishment.reindex(columns=list(range(1001, 4001)))
date_of_establishment

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
成立日期,-9633,-6741,-7167,-7811,-7126,-7692,-7189,-9014,-8237,-9638,...,-5474,-6269,-6045,-7151,-8616,-7756,-7665,-6858,-5244,-6411


In [9]:
business_status = pd.get_dummies(df['经营状态'], prefix='经营状态').groupby('企业编号').mean().T
business_status = business_status.reindex(columns=list(range(1001, 4001))).fillna(0)
business_status

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
经营状态_存续（在营、开业、在册）,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
经营状态_迁出,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
industry_category_code = pd.get_dummies(df['行业大类（代码）'], prefix='行业大类（代码）').groupby('企业编号').mean().T
industry_category_code = industry_category_code.reindex(columns=list(range(1001, 4001))).fillna(0)
industry_category_code

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
行业大类（代码）_A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_C,0,0,1,0,0,0,1,0,0,0,...,0,1,1,1,0,1,0,0,1,1
行业大类（代码）_D,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
行业大类（代码）_N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_O,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业大类（代码）_Q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
industry_subclass_code = pd.get_dummies(df['行业小类（代码）'], prefix='行业小类（代码）').groupby('企业编号').mean().T
industry_subclass_code = industry_subclass_code.reindex(columns=list(range(1001, 4001))).fillna(0)
industry_subclass_code

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
行业小类（代码）_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
行业小类（代码）_86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
行业小类（代码）_89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
types = pd.get_dummies(df['类型'], prefix='类型').groupby('企业编号').mean().T
types = types.reindex(columns=list(range(1001, 4001))).fillna(0)
types

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
类型_外企,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
类型_民营,1,1,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1


In [13]:
provincial_code = pd.get_dummies(df['省份代码'], prefix='省份代码').groupby('企业编号').mean().T
provincial_code = provincial_code.reindex(columns=list(range(1001, 4001))).fillna(0)
provincial_code

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
省份代码_10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_11,0,0,0,1,1,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
省份代码_12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
省份代码_61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
省份代码_64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
city_code = pd.get_dummies(df['城市代码'], prefix='城市代码').groupby('企业编号').mean().T
city_code = city_code.reindex(columns=list(range(1001, 4001))).fillna(0)
city_code

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
城市代码_0,1,1,1,0,0,0,1,1,1,1,...,0,1,0,1,0,0,0,1,0,0
城市代码_1,0,0,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
城市代码_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
城市代码_3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,0,0,1,0
城市代码_4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
城市代码_34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
城市代码_42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
城市代码_51,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
城市代码_52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
whether_it_is_listed = pd.get_dummies(df['是否上市'], prefix='是否上市').groupby('企业编号').mean().T
whether_it_is_listed = whether_it_is_listed.reindex(columns=list(range(1001, 4001))).fillna(0)
whether_it_is_listed

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
是否上市_N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
是否上市_Y,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [16]:
operating_period_from = df[['经营期限自']].applymap(to_day).groupby('企业编号').mean().T
operating_period_from = operating_period_from.reindex(columns=list(range(1001, 4001)))
operating_period_from

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
经营期限自,-9633.0,-6741.0,-7167.0,-7811.0,-7126.0,-7692.0,-7189.0,-9014.0,-8237.0,-9638.0,...,-5474.0,-6269.0,-6045.0,-7151.0,-8616.0,-7756.0,-7665.0,-6858.0,-5244.0,-6411.0


In [18]:
date_of_issue = df[['发照日期']].applymap(to_day).groupby('企业编号').mean().T
date_of_issue = date_of_issue.reindex(columns=list(range(1001, 4001)))
date_of_issue

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
发照日期,-9633.0,-439.0,-134.0,-14.0,-33.0,-7692.0,-278.0,-190.0,-908.0,-144.0,...,-690.0,-156.0,-88.0,-837.0,-97.0,-98.0,-31.0,-237.0,-615.0,-931.0


In [24]:
number_of_workers = df[['员工人数']].groupby('企业编号').mean().T
number_of_workers = number_of_workers.reindex(columns=list(range(1001, 4001)))
number_of_workers

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
员工人数,19.0,12.0,11.0,15.0,15.0,14.0,63.0,22.0,12.0,29.0,...,13.0,10.0,13.0,12.0,11.0,9.0,14.0,10.0,12.0,10.0


In [32]:
reason_for_cancellation = pd.get_dummies(df['注销原因'], prefix='注销原因').groupby('企业编号').mean().T
reason_for_cancellation = reason_for_cancellation.reindex(columns=list(range(1001, 4001))).fillna(0)
reason_for_cancellation

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
注销原因_其他原因,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
注销原因_决议解散,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
注销原因_因公司合并或分立,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
注销原因_宣告破产,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
注销原因_章程规定的解散事由出现,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
pd.concat([registered_capital_ten_thousand_yuan, registered_capital_currency_regular, date_of_establishment, 
               business_status, industry_category_code, industry_category_code, types, provincial_code, city_code, 
               whether_it_is_listed, operating_period_from, date_of_issue, number_of_workers, 
               reason_for_cancellation]).to_pickle('工商基本信息表.pickle')